# 1.2 Reading a 2D vector field from a simulation

In [ ]:
%matplotlib inline
import sys
import os

# Sort the paths out to run from this file
parent_path = os.path.dirname(os.getcwd())
src_path = os.path.join(os.path.sep,parent_path, 'src')
sys.path.append(src_path)
print(src_path)

## Reading 2D field

list

In [ ]:
from sim_reader import MeshReader, Unfolder
from fields import ScalarField, VectorField
from object_reader import PickleManager
from regressors import LModel, CSModel
import numpy as np


# Define the necessary objects
reader = MeshReader('monoblock_out.e')
pickle_manager = PickleManager()
unfolder = Unfolder()

# Read the simulation data
sensor_region = 'right'
pos_3D = reader.read_pos(sensor_region)
pos_2D = unfolder.compress_2D(pos_3D)
bounds = unfolder.find_bounds(pos_2D)
grid = unfolder.generate_grid(bounds, 30, 30)

disp = np.array([
    reader.read_scalar(sensor_region, 'disp_x'),
    reader.read_scalar(sensor_region, 'disp_y'),
    reader.read_scalar(sensor_region, 'disp_z')
]).T

disp_field = VectorField(LModel, bounds)
disp_field.fit_model(pos_2D, disp)

# Save the simulation data
pickle_manager.save_file('simulation', 'disp_plane_field.obj', disp_field)
pickle_manager.save_file('simulation', 'disp_plane_points.obj', grid)

## Squashing to line

description

In [ ]:
# Convert the 2D field to a 1D line
pos1 = (bounds[0][0], bounds[0][1])
pos2 = (bounds[0][0], bounds[1][1])
line_2D = unfolder.generate_line(pos1, pos2, 50)
line_disps = disp_field.predict_values(line_2D)
line_1D = unfolder.compress_1D(line_2D)

bounds_1D = np.array([line_1D[0], line_1D[-1]])
new_line_field = VectorField(CSModel, bounds_1D)
new_line_field.fit_model(line_1D, line_disps)

# Save the new 1D line field
pickle_manager.save_file('simulation', 'disp_line_field.obj', new_line_field)
pickle_manager.save_file('simulation', 'disp_line_points.obj', line_1D)